Importing necessary libraries 

In [ ]:
!pip install yake
!pip install stanza

import nltk, spacy, csv, re, yake, stanza, json
import pandas as pd
import numpy as np

from google.colab import drive
from datetime import datetime
from pprint import pprint
import tensorflow as tf


nltk.download('stopwords')
nltk.download('punkt')
stanza.download('en')

nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
kw_extractor = yake.KeywordExtractor()

drive.mount('/content/drive')

Dataset preprocessing

In [ ]:
with open('') as f:
    json_data = json.load(f)

data= []
for x in range(0, 150):
  first_key = list(json_data.values())[x]

  try:
    first_ = list(first_key.values())[1]
  except IndexError:
    first_ = list(first_key.values())[0]
  
  str_ = ''
  if first_[7]:
    for row in first_[7]:
      if '{' in row[0]:
        s = row[0].split('{', 1)[0]
        str_ = str_ + ' | ' + s
      else:
        str_ = str_ + ' | ' + row[0] + ' | ' + row[4]
    data.append(str_)

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data)
seq = tokenizer.texts_to_sequences(data)

X = []
y = []
total_words_dropped = 0

for i in seq:
    if len(i) > 1:
        for index in range(1, len(i)):
            X.append(i[:index])
            y.append(i[index])
    else:
        total_words_dropped += 1

Training the recommender model

In [ ]:
X = tf.keras.preprocessing.sequence.pad_sequences(X)
y = tf.keras.utils.to_categorical(y)

vocab_size = len(tokenizer.word_index) + 1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 14),
    tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(vocab_size, activation='softmax'),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.004), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

model.fit(X, y, epochs=100)
model.save('nwp.h5')

In [ ]:
model.summary()